In [1]:
from databricks.connect import DatabricksSession

spark = DatabricksSession.builder.profile("DEFAULT").getOrCreate()

from databricks.sdk import WorkspaceClient
w = WorkspaceClient(profile = "DEFAULT")
dbutils = w.dbutils

dbutils.fs.mkdirs('/Volumes/emissions_datapipeline_workspace/default/datalake/')
dbutils.fs.mkdirs('/Volumes/emissions_datapipeline_workspace/default/datalake/raw')
dbutils.fs.mkdirs('/Volumes/emissions_datapipeline_workspace/default/datalake/curated')
dbutils.fs.mkdirs('/Volumes/emissions_datapipeline_workspace/default/datalake/serving')


display(dbutils.fs.ls('/Volumes/emissions_datapipeline_workspace/default/datalake'))


[FileInfo(path='/Volumes/emissions_datapipeline_workspace/default/datalake/curated/', name='', size=None, modificationTime=None),
 FileInfo(path='/Volumes/emissions_datapipeline_workspace/default/datalake/raw/', name='', size=None, modificationTime=None),
 FileInfo(path='/Volumes/emissions_datapipeline_workspace/default/datalake/serving/', name='', size=None, modificationTime=None)]

In [2]:
from datetime import datetime
import pathlib

current_date = datetime.today().strftime('%Y%m%d')
current_path = pathlib.Path().resolve()
local_path = f'{current_path}/data/world_bank_data/'
dbfs_path = f'/Volumes/emissions_datapipeline_workspace/default/datalake/raw/world_development_indicators/date={current_date}/'

In [ ]:
# Use dbutils to copy local files to databricks volume

dbutils.fs.cp(
  f'file:{local_path}', 
  f'dbfs:/Volumes/emissions_datapipeline_workspace/default/datalake/raw/world_development_indicators/date={current_date}/', 
  recurse=True
)

In [ ]:
# Use spark instead of dbutils to load files
# import os
# 
# csv_files = [file for file in os.listdir(local_path) if file.endswith(".csv")]
# 
# for csv_file in csv_files:
#     file_path = os.path.join(local_path, csv_file)
#     
#     df = spark.read.format("csv").option("header", "true").load(file_path)
#     
#     # Write DataFrame to DBFS
#     dbfs_file_path = os.path.join(dbfs_path, csv_file)
#     df.write.format("csv").mode("overwrite").save(dbfs_file_path)
#     
#     print(f"Successfully copied {csv_file} to DBFS.")

In [ ]:
for year in [2017, 2018, 2019]:
  file_name = f'co2_emissions_passenger_cars_{year}.json'
  dbutils.fs.cp(
    f'file:{current_path}/data/emissions_data/{file_name}', 
    f'dbfs:/Volumes/emissions_datapipeline_workspace/default/datalake/raw/co2_passenger_cars_emissions/year={year}/{file_name}'
  )

In [13]:
df_co2_emissions                = (spark.read
                                        .option("multiline","true")
                                        .json('dbfs:/Volumes/emissions_datapipeline_workspace/default/datalake/raw/co2_passenger_cars_emissions'))
df_world_development_indicators = (spark.read
                                        .option('header', 'true')
                                        .csv(f'dbfs:/Volumes/emissions_datapipeline_workspace/default/datalake/raw/world_development_indicators/date={current_date}/world_bank_data/WDIData.csv'))

# Printing the schemas of the two dataframes
df_co2_emissions.printSchema()
df_world_development_indicators.printSchema()

root
 |-- At1 (mm): long (nullable = true)
 |-- At2 (mm): long (nullable = true)
 |-- Cn: string (nullable = true)
 |-- Cr: string (nullable = true)
 |-- Ct: string (nullable = true)
 |-- De: double (nullable = true)
 |-- E (g/km): string (nullable = true)
 |-- Enedc (g/km): long (nullable = true)
 |-- Er (g/km): string (nullable = true)
 |-- Ernedc (g/km): double (nullable = true)
 |-- Erwltp (g/km): double (nullable = true)
 |-- Ewltp (g/km): long (nullable = true)
 |-- Fm: string (nullable = true)
 |-- Ft: string (nullable = true)
 |-- ID: long (nullable = true)
 |-- It: string (nullable = true)
 |-- MMS: string (nullable = true)
 |-- MS: string (nullable = true)
 |-- Man: string (nullable = true)
 |-- Mh: string (nullable = true)
 |-- Mk: string (nullable = true)
 |-- Mp: string (nullable = true)
 |-- Mt: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- T: string (nullable = true)
 |-- TAN: string (nullable = true)
 |-- VFN: string (nullable = true)
 |-- Va: string

In [14]:
print(f"Number of records  for CO2 emissions DF: {df_co2_emissions.count()}")
print(f"Number of records  for World Development Indicators: {df_world_development_indicators.count()}")

Number of records  for CO2 emissions DF: 300000
Number of records  for World Development Indicators: 383838


In [15]:
display(df_co2_emissions.describe())

,summary,At1 (mm),At2 (mm),Cn,Cr,Ct,De,E (g/km),Enedc (g/km),Er (g/km),Ernedc (g/km),Erwltp (g/km),Ewltp (g/km),Fm,Ft,ID,It,MMS,MS,Man,Mh,Mk,Mp,Mt,Status,T,TAN,VFN,Va,Ve,Vf,W (mm),Zr,ec (cm3),ep (KW),m (kg),r,version_file,year,z (Wh/km)
0,count,283776,276247,299985,299094,298302,10068,0,300000,0,7155,2814,109321,299993,300000,300000,275357,299262,299996,300000,300000,299782,300000,109779,300000,299974,299872,285563,299901,299844,16254,286685,0,299975,248605,299434,300000,300000,300000,57
1,mean,1630.8991317095174,1637.648636908274,800.1971752921536,None,None,-0.0016828168454510464,None,265.74504333333334,None,1.6373025856043493,0.019189765458422166,307.63519360415654,None,None,5457517.07155,7.285714285714286,8.0,None,None,None,8.0,None,2197.6191348072034,None,474.42689563652726,111.0,None,3.1930333817126228E90,322978.8396673288,0.25599852344038393,2785.987331042782,None,4194.065415451288,340.59067194947806,2059.734789636447,2.302503333333333,None,2018.0,217.24561403508773
2,stddev,57.18203871006447,53.35806895019124,948.4930169775346,None,None,0.040810453353788106,None,46.160696807975754,None,0.3011054880652157,0.18632959405329533,36.00059485372222,None,None,3902514.363239479,3.5326937743674756,0.0,None,None,None,0.0,None,517.6607731000473,None,5658.785598087578,128.17175976009693,None,3.349852913181437E91,684271.5794259827,0.43643441442040254,320.5153047443445,None,1161.4346709683432,107.19919141481658,352.7646953486153,21.99410089550318,None,0.816497941758763,21.869973464738614
3,min,501,501,,,,-0.078,None,185,None,1.0,0.0,11,,,307,,,AT,,AA-IVA,,,0,F,,,,,,0,0,None,898,27,242,1,v15,2017,100
4,max,2895,2250,unknown,m1,m1,2.0,None,671,None,3.0,1.9,535,P,petrol/electric,15499148,e24*2,ZESZUTA,SK,VOLVO CAR CORPORATION,unknown,ZENVO,VW GROUP PC,4612,P,l50,e9*ks07/46*6716*01,RL-ZE1AEM57A6_0000-SJN-1,ZYX10(H),h,1,4762,None,8382,1103,4160,3250,v20,2019,296


In [16]:
display(df_world_development_indicators.describe())

,summary,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,_c65
0,count,383838,383838,383838,383838,37535,42462,44706,44546,45098,47297,47381,49105,48666,50502,69180,76155,79268,78762,80277,83976,86306,90294,89880,90735,95682,98055,99788,99293,100047,101427,102269,103907,102374,104236,127130,134506,139588,140165,142471,150215,150058,152364,153543,159107,184652,178983,184432,184143,189666,200501,201218,206398,204140,205281,219195,211412,216002,210128,217378,212122,212169,208500,201189,168957,73036,0
1,mean,None,None,None,None,2.592876336432356E11,2.4779261353108875E11,2.5013558081062997E11,2.590327127379009E11,2.762048061792987E11,2.9906942373923914E11,3.21279135202025E11,3.384774628167722E11,3.906542640654587E11,4.2261796999330286E11,3.7334095502703815E11,3.8032528473845306E11,4.147666701615174E11,4.691563606027701E11,5.647158949644286E11,5.72076534234965E11,6.322169491890945E11,6.127279472091635E11,5.710701151433666E11,5.542882655022534E11,4.849080205790727E11,4.822868032087772E11,5.1440310422167804E11,6.912671886125896E11,7.564952778023131E11,8.665241439542793E11,8.351603971349722E11,7.953259975687358E11,6.183742051454916E11,6.824146791377161E11,7.296705558181024E11,7.019809871180205E11,7.182135580428477E11,7.935252791093323E11,8.90554524984262E11,9.182036456669215E11,8.859332770238416E11,8.990489784052509E11,8.139973438437421E11,7.97168392629255E11,7.248125842085822E11,7.809091388333547E11,9.448284931831068E11,9.152183488111382E11,1.053976340116476E12,1.0781681215247091E12,1.2152049249672415E12,1.448502151714482E12,1.6501618926697764E12,1.7198605070977822E12,1.9151103842494346E12,2.2259232557435815E12,2.3569779336184224E12,2.7042589895662744E12,2.856930714972209E12,3.049426845683234E12,3.311004239528729E12,3.5275280213165835E12,4.0649971020089727E12,5.333820836933096E12,1.2581113484111328E13,None
2,stddev,None,None,None,None,1.0797733738476117E13,1.1147454676350646E13,1.1630259678816145E13,1.2239266006399559E13,1.3185972623906729E13,1.4755976342961E13,1.6251153807308844E13,1.774356643503081E13,2.0395801001277594E13,2.3088122782073535E13,2.1965429832077758E13,2.3866724299488383E13,2.68029402281783E13,2.9520474293516625E13,3.3410995460145758E13,3.3339841929436875E13,3.825239768528878E13,3.652815766711666E13,3.1790837897702965E13,2.921166290799741E13,2.382301806352306E13,2.3176993694507754E13,2.5873541955899145E13,4.011283619004738E13,5.051901913685053E13,6.747023298969066E13,6.4298326730174875E13,5.423488473196773E13,2.8442286210096742E13,3.125915911819643E13,4.0879901620072984E13,3.5217592709358625E13,3.688952055006893E13,4.327235280941269E13,5.180829957928165E13,5.3962475114428336E13,4.001425563622181E13,3.828414793883204E13,3.41579013022995E13,3.657932578580088E13,3.9897309180472484E13,4.61948642829912E13,3.8057211801649234E13,5.190882012551103E13,4.501172040073363E13,4.930039316540269E13,5.178744246257089E13,5.585828029423742E13,6.182191329194331E13,6.569723687493841E13,7.635823843076386E13,8.733411478914084E13,9.384930967336153E13,1.0860499402777803E14,1.1878605191743314E14,1.259575467244428E14,1.3945529413647869E14,1.510028286231329E14,1.7636759565107884E14,2.247591722133191E14,4.080741502079425E14,None
3,min,Afghanistan,ABW,ARI treatment (% of children under 5 taken to a health provider),AG.AGR.TRAC.NO,-0.00330839030000001,-0.00172840179796196,-0.000155636363545455,-0.000243272726363636,-0.000649454545454545,-0.000641454544454546,-0.000554181818454545,-0.000876727273454545,-0.0002550001,-0.0010000001,-0.0002970001,-0.0010000001,-0.000269999999999997,-0.000459999996666664,-0.00049591064453125,-0.000168580785369902,-0.000101566299996136,-0.000157833,-0.00013113,-0.000151525860840636,-0.000214309693232913,-0.00018310

In [17]:
display(df_co2_emissions)

,At1 (mm),At2 (mm),Cn,Cr,Ct,De,E (g/km),Enedc (g/km),Er (g/km),Ernedc (g/km),Erwltp (g/km),Ewltp (g/km),Fm,Ft,ID,It,MMS,MS,Man,Mh,Mk,Mp,Mt,Status,T,TAN,VFN,Va,Ve,Vf,W (mm),Zr,ec (cm3),ep (KW),m (kg),r,version_file,year,z (Wh/km)
0,NaN,NaN,BUGATTI BG 744,M1,M1,NaN,None,575,None,NaN,NaN,NaN,M,petrol,4169823,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,NaN,F,,,,,,NaN,NaN,None,7993.0,1103.0,1995,1,v20,2019,NaN
1,NaN,NaN,BUGATTI CHIRON-DIVO,M1,M1,NaN,None,575,None,NaN,NaN,NaN,M,petrol,4169522,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,NaN,P,,,,,,NaN,NaN,None,NaN,1103.0,1995,1,v19,2019,NaN
2,NaN,NaN,BUGATTI CHIRON-DIVO,M1,M1,NaN,None,575,None,NaN,NaN,NaN,M,petrol,4169522,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,NaN,F,,,,,,NaN,NaN,None,NaN,1103.0,1995,1,v20,2019,NaN
3,NaN,NaN,BUGATTI BG 744,M1,M1,NaN,None,575,None,NaN,NaN,NaN,M,petrol,4169823,,BUGATTI F,DE,AA-IVA,AA-IVA,BUGATTI,,NaN,P,,,,,,NaN,NaN,None,7993.0,1103.0,1995,1,v19,2019,NaN
4,NaN,NaN,AUDI A8,M1,M1,NaN,None,545,None,NaN,NaN,NaN,M,petrol,4784577,,AUDI,DE,AA-IVA,AA-IVA,AUDI,,NaN,F,F8,,,,,NaN,NaN,None,3993.0,420.0,3895,1,v20,2019,NaN
5,NaN,NaN,AUDI A8,M1,M1,NaN,None,545,None,NaN,NaN,NaN,M,petrol,4784577,,AUDI,DE,AA-IVA,AA-IVA,AUDI,,NaN,P,F8,,,,,NaN,NaN,None,3993.0,420.0,3895,1,v19,2019,NaN
6,1747.0,1670.0,,M1,M1,NaN,None,516,None,NaN,NaN,505.0,M,petrol,754447,,BUGATTI,GB,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150.0,P,5B,e1*ks07/46*0008*09,,DALAADX1CN,TAD7AD71C002,NaN,2711.0,None,7993.0,NaN,2070,1,v19,2019,NaN
7,1747.0,1670.0,BUGATTI CHIRON,M1,M1,NaN,None,516,None,NaN,NaN,506.0,M,petrol,4898524,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150.0,P,5B,E1*KS07/46*0008*09,,DALAADX1CN,TAD7AD71C002,NaN,2711.0,None,7993.0,1103.0,2070,1,v19,2019,NaN
8,1747.0,1670.0,BUGATTI CHIRON,M1,M1,NaN,None,516,None,NaN,NaN,506.0,M,petrol,4898531,,BUGATTI F,DE,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150.0,F,5B,E1*KS07/46*0008*09,,DALAADX1CN,TAD7AD71C002,NaN,2711.0,None,7993.0,1103.0,2070,1,v20,2019,NaN
9,1747.0,1670.0,,M1,M1,NaN,None,516,None,NaN,NaN,505.0,M,petrol,12889810,,BUGATTI,GB,BUGATTI AUTOMOBILES SAS,BUGATTI,BUGATTI,VW GROUP PC,2150.0,P,5B,e1*ks07/46*0008*09,,DALAADX1CN,TAD7AD71C002,NaN,2711.0,None,7993.0,NaN,2070,1,v19,2019,NaN


In [18]:
display(df_world_development_indicators)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,_c65
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for cooking (% of population),EG.CFT.ACCS.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,12.205985334256,12.5493332143826,12.8877052744847,13.2263347774934,13.5755915831281,13.9243539706952,14.2660310520972,14.5962971181626,14.9552860566119,15.2810233296453,15.6312911024101,15.9812561302977,16.320474865037,16.6432428558172,16.9946949543723,17.3131682773135,17.6049500630141,None,None,None,None,None
1,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,20.0860066130196,23.1031784776438,24.2234301519858,25.0231841252266,26.1763357723345,23.621498439778,27.2994330458403,28.737952342644,28.813735402838,27.4830612476805,28.2333728874701,28.3818359876251,32.2240274434276,32.0464780952398,31.3235788003652,33.3121631315881,38.3804331871331,39.754201347135,42.1682410699846,43.6406611120552,None,None
2,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural population)",EG.ELC.ACCS.RU.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,8.49160997191625,13.2018746955682,15.2093473536694,13.8135351072229,15.4236146358427,10.9588403149893,16.7624691704676,14.417101517166,16.9779753635741,18.1733410522112,16.2410638636384,15.2959503612632,20.5253527781327,19.4613833438545,17.790698468584,16.5534701416911,23.9078966279348,24.6247249863346,26.8138996635713,28.8411502327974,None,None
3,Africa Eastern and Southern,AFE,"Access to electricity, urban (% of urban population)",EG.ELC.ACCS.UR.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,56.0834516422826,54.7701226470266,56.1023300404056,57.0943101230703,57.836866299283,58.9533652971298,59.5469745890136,59.5989593932235,60.9834004395019,61.2272550575374,62.5208242745159,65.4706154133,66.303599202538,66.4960101543106,65.8289881700667,66.9266915686378,68.7221844313438,71.0854179924979,71.9949328691005,73.5898862435969,None,None
4,Africa Eastern and Southern,AFE,Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),FX.OWN.TOTL.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,Africa Eastern and Southern,AFE,"Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)",FX.OWN.TOTL.FE.ZS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,Africa Eastern and Southern,AFE,"Account ownership at a financial institution or with a mobile-money-ser